In [11]:
from tensorflow.keras.models import load_model
import pickle
from albumentations import ShiftScaleRotate,Cutout,RandomContrast,RandomBrightness,Compose
from utility.albumentations_helper import create_transform
from functions import generate_feature,loop_distance,l2_distance_np,depthwise_maker,top_k
import numpy as np
import pandas as pd
from functools import reduce,partial

In [69]:
## change ##
color = False
FFA_size = 4
pct = 0.09
model_file = 'feature_model_DenseNet_Triplet2'
pos_str = '_new_whale'
#distanceFun = depthwise_maker(np.load('Models/conv_weight_Depthwise2.npy'))
distanceFun = l2_distance_np
aggFun = partial(np.quantile,q=pct,axis=(1,2))
new_whale_count = 2197

Load model

In [3]:
feature_model = load_model('Models/'+model_file+'.h5')

In [4]:
feature_model.compile(loss='mse',optimizer='SGD')

In [6]:
#feature_model = load_model('Models/feature_l2_distance_margin_loss_fun_factory(0.1,5).h5', \
#                            custom_objects={'loss': margin_loss_fun_factory(0.1,5)})

In [5]:
with open('submit_train.pkl', 'rb') as f:
    submit_train = pickle.load(f)
with open('submit_test.pkl', 'rb') as f:
    submit_test = pickle.load(f)  
with open('submit_map.pkl', 'rb') as f:
    submit_map = pickle.load(f)        

In [8]:
aug = Compose([RandomContrast(p=0.2),RandomBrightness(p=0.2),
                ShiftScaleRotate(shift_limit=0.03,rotate_limit=15,scale_limit=0.02,p=1)])
transform = create_transform(aug)    

In [9]:
feature_train = generate_feature(submit_train,transform,FFA_size,color,feature_model)
feature_val = generate_feature(submit_test,transform,FFA_size,color,feature_model)

In [16]:
feature_train = feature_train[:,:,np.newaxis,:]
distance_list = []
index_list = []
for feature in feature_val:
    feature = feature[np.newaxis,np.newaxis,:,:]
    d = distanceFun(feature_train,feature)
    distance_list.append(d)
    index_list.append(top_k(aggFun(d)))
distance_matrix = np.array(distance_list)
predicts = np.array(index_list)

In [104]:
d_diag = distanceFun(feature_train,np.swapaxes(feature_train,1,2))
d_diag = np.reshape(d_diag,(d_diag.shape[0],-1))
d_diag = np.sort(d_diag,1)
d_diag = np.min(d_diag[:,4:],1)

In [105]:
distance_matrix = distance_matrix - d_diag[np.newaxis,:,np.newaxis,np.newaxis]

In [106]:
distance_matrix = np.reshape(distance_matrix,(distance_matrix.shape[0],-1))

In [107]:
distance_matrix = np.quantile(distance_matrix,100/distance_matrix.shape[1],1)

In [108]:
newIndex = distance_matrix > np.quantile(distance_matrix,1-2*new_whale_count/feature_val.shape[0])

In [109]:
newIndex = newIndex[:,np.newaxis]

In [111]:
predicts = np.vectorize(submit_map.get)(predicts)

In [112]:
predicts_new = np.concatenate([np.array([['new_whale']]*feature_val.shape[0]),predicts[:,:4]],1)

In [113]:
predicts_final = np.where(newIndex,predicts_new,predicts)

In [114]:
predicts_final=reduce(np.core.defchararray.add,[predicts_final[:,0],\
                                                 ' ', predicts_final[:,1],\
                                                 ' ', predicts_final[:,2],\
                                                 ' ', predicts_final[:,3],\
                                                 ' ', predicts_final[:,4]])

In [115]:
submit_df = pd.DataFrame(list(zip([i[0].split('/')[-1].split('.')[0]+'.jpg' for i in submit_test],\
                          predicts_final.tolist())), columns=['Image','Id'])

In [117]:
submit_df.to_csv('submissions/'+model_file+pos_str+'.csv',index=False)